In [13]:
import pandas as pd
import pandasql as ps
from pandasql import sqldf
import datetime
import os

In [14]:
filepath = "C:/Users/spevans/OneDrive - Sopra Steria/Projects/NS&I"

In [15]:
lst_parquet_files = os.listdir(filepath +'/mockfiles')
lst_parquet_files,len(lst_parquet_files)

(['API_Logs.parquet',
  'audits.parquet',
  'availability.parquet',
  'employees.parquet',
  'message_information.parquet',
  'multi_availability.parquet',
  'multi_response.parquet',
  'nicecxone_callanalytics.parquet',
  'nicecxone_customersatisfaction.parquet',
  'postout_bulk.parquet',
  'ppa_cancellationnotification.parquet',
  'request_information.parquet',
  'servicenow_applicationcancellation.parquet',
  'servicenow_applicationrejection.parquet',
  'servicenow_bonding.parquet',
  'servicenow_caseclosed.parquet',
  'servicenow_casecreated.parquet',
  'servicenow_casereopened.parquet',
  'servicenow_changeincidents.parquet',
  'servicenow_changerecordclosed.parquet',
  'servicenow_changerecordopened.parquet',
  'servicenow_changerecordupdated.parquet',
  'servicenow_complaintresponse.parquet',
  'servicenow_deployedfixes.parquet',
  'servicenow_firstcontactresolution.parquet',
  'servicenow_foscomplaint.parquet',
  'servicenow_incidentclosed.parquet',
  'servicenow_incidentopened

In [16]:
## calling in all dataframes

df_audits = pd.read_parquet(filepath+'/mockfiles/audits.parquet')
df_availability = pd.read_parquet(filepath+'/mockfiles/availability.parquet')


In [17]:
df_availability.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   call_id            300 non-null    object        
 1   call_system        300 non-null    object        
 2   call_response      300 non-null    object        
 3   created_timestamp  300 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(3)
memory usage: 9.5+ KB


In [18]:
## converting the date column to proper format

df_audits['date_raised']=pd.to_datetime(df_audits['date_raised'], infer_datetime_format=True)
df_audits['resolution_due_date']=pd.to_datetime(df_audits['resolution_due_date'], infer_datetime_format=True)

df_availability['created_timestamp']=pd.to_datetime(df_availability['created_timestamp'],infer_datetime_format=True)


In [19]:
df_audits.to_parquet('C:/Users/spevans/OneDrive - Sopra Steria/Projects/NS&I/mockfiles/audits.parquet')
df_availability.to_parquet('C:/Users/spevans/OneDrive - Sopra Steria/Projects/NS&I/mockfiles/availability.parquet')

### KPI 7.4
Audit and Compliance Actions
Open actions related to audit and compliance. 

#### count records where todaydate - resolution_due_date > 30 days

In [22]:
df_audits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   audit_id             300 non-null    object        
 1   date_raised          300 non-null    datetime64[ns]
 2   resolution_due_date  300 non-null    datetime64[ns]
dtypes: datetime64[ns](2), object(1)
memory usage: 7.2+ KB


In [31]:
sql_query = """

SELECT COUNT(*) AS open_actions
FROM df_audits
WHERE julianday('now') - julianday(resolution_due_date) > 30;

"""

print(sqldf(sql_query, locals()))

   open_actions
0           115


### KPI 9.1
Availability 
Documents View  
The availability of the Documents viewing services to enable Customers, users and other services to access and view documents.

#### for today-30 to today
#### where call_system = Alfresco
#### s1. count records
#### s2. count records where call_response = 'Positive'
#### s3. s2/s1*100

In [32]:
df_availability

,call_id,call_system,call_response,created_timestamp
0,CALL001,Alfresco,Negative,2024-01-01 14:49:00
1,CALL002,Alfresco,Negative,2024-02-01 14:33:00
2,CALL003,Poly AI,Negative,2024-02-01 17:44:00
3,CALL004,Alfresco,Neutral,2024-03-01 10:54:00
4,CALL005,Poly AI,Neutral,2024-03-01 13:03:00
...,...,...,...,...
295,CALL296,Alfresco,Neutral,2024-06-29 10:04:00
296,CALL297,Poly AI,Negative,2024-06-29 14:50:00
297,CALL298,Poly AI,Positive,2024-06-29 17:56:00
298,CALL299,Alfresco,Positive,2024-06-30 10:05:00


In [38]:
sql_query = """

WITH selected_df AS (
    SELECT *
    FROM df_availability
    WHERE call_system ="Alfresco" 
    AND (created_timestamp BETWEEN DATETIME('NOW','-30 days') AND DATETIME('NOW'))
)
,
agg_results AS (
SELECT COUNT(*) AS Total
,SUM(CASE WHEN call_response = 'Positive' THEN 1 ELSE 0 END) AS positive_call_response
FROM selected_df
)

SELECT (positive_call_response * 1.0 / Total) * 100 AS positive_count
FROM agg_results

"""
print(sqldf(sql_query, locals()))

   positive_count
0           31.25
